In [ ]:
from kafka import KafkaConsumer
import pandas as pd
# connect to kafka server
consumer3 = KafkaConsumer('airpollution_from_sensor', bootstrap_servers=['localhost:9094'], auto_offset_reset='latest')
from datetime import datetime
import time
# subscribe to the topic and save the data to a csv file
curr_time = datetime.now()
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'])
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS airpollution 
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")
session.set_keyspace('airpollution')
session.execute("""
        CREATE TABLE IF NOT EXISTS sensor_data (
            locationName text,
            timestamp bigint,
            pm02 float,
            atmp float,
            rhum float,
            rco2 float,
            PRIMARY KEY (locationName, timestamp)
        )
    """)

while True:
    from cassandra.cluster import Cluster
    cluster = Cluster(['localhost'])
    session = cluster.connect('airpollution')
    import json
    for message in consumer3:
        print(f"Message received: {message.value}")
        data = {}
        message_value = json.loads(message.value.decode('utf-8'))
        data['locationName'] = message_value['locationName']
        data['timestamp'] = message_value['timestamp']
        data['pm02'] = message_value['pm02']
        data['atmp'] = message_value['atmp']
        data['rhum'] = message_value['rhum']
        data['rco2'] = message_value['rco2']
        print(f"Data to insert: {data}")
        df = pd.DataFrame(data, index=[0])
        df.to_csv('airpollution_sensor.csv', mode='a', header=False, index=False)
        print("Inserting data into Cassandra...")
        session.execute(
            """
            INSERT INTO sensor_data (locationName, timestamp, pm02, atmp, rhum, rco2)
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (data['locationName'], int(data['timestamp']), data['pm02'], data['atmp'], data['rhum'], data['rco2'])
        )
        
    